In [1]:
import json, os
from pathlib import Path

In [2]:
dataset_path = "/data/falcetta/Job-VS-TOPCOW/ORIGINAL/"
json_dir = '/home/falcetta/0_PhD/JOB-VS/JoB-VS/datasets/TOPCOW'
json_out_filepath = os.path.join(json_dir, 'dataset.json')
Path(json_dir).mkdir(parents=True, exist_ok=True)

train_folder = os.path.join(dataset_path, "train")
test_folder = os.path.join(dataset_path, "test")

assert os.path.exists(train_folder), "Train folder not found"
assert os.path.exists(test_folder), "Test folder not found"

### Generage Dataset json file

get_id function working


In [4]:
def extract_paired_list(dataset_complete_path, dataset_name=''):
    
    assert os.path.exists(dataset_complete_path), f"{dataset_name} folder not found"
    assert dataset_name in ['train', 'test'], "dataset_name must be 'train' or 'test'"
    
    data_list = os.listdir(dataset_complete_path)
    data_list = [os.path.join(dataset_name, x) for x in data_list]
    data_img_list = sorted([x for x in data_list if 'ToF' in x])
    data_label_list = sorted([x for x in data_list if 'vessel' in x])

    assert len(data_img_list) == len(data_label_list), "Images and labels have different length"
    print(f"Dataset ({dataset_name}) size: {len(data_label_list)}") 


    for img, label in zip(data_img_list, data_label_list):
        assert get_id(img) == get_id(label), "Images and labels are not aligned"

    return data_img_list, data_label_list

train_img_list, train_label_list = extract_paired_list(train_folder,'train')
test_img_list, test_label_list = extract_paired_list(test_folder, 'test')
    
val_img_list = []
val_label_list = []


Dataset (train) size: 77
Dataset (test) size: 13


Validation set size: 13

Val images: ['train/020_ToF.nii.gz', 'train/005_ToF.nii.gz', 'train/048_ToF.nii.gz', 'train/042_ToF.nii.gz', 'train/039_ToF.nii.gz', 'train/026_ToF.nii.gz', 'train/021_ToF.nii.gz', 'train/090_ToF.nii.gz', 'train/018_ToF.nii.gz', 'train/076_ToF.nii.gz', 'train/007_ToF.nii.gz', 'train/006_ToF.nii.gz', 'train/024_ToF.nii.gz']
Val labels: ['train/020_vessel_mask.nii.gz', 'train/005_vessel_mask.nii.gz', 'train/048_vessel_mask.nii.gz', 'train/042_vessel_mask.nii.gz', 'train/039_vessel_mask.nii.gz', 'train/026_vessel_mask.nii.gz', 'train/021_vessel_mask.nii.gz', 'train/090_vessel_mask.nii.gz', 'train/018_vessel_mask.nii.gz', 'train/076_vessel_mask.nii.gz', 'train/007_vessel_mask.nii.gz', 'train/006_vessel_mask.nii.gz', 'train/024_vessel_mask.nii.gz']


In [6]:
num_train, num_val, num_test = len(train_img_list), len(val_img_list), len(test_img_list)
print(f"Train set size: {num_train} ({num_train/(num_train+num_val+num_test)*100:.2f}%)")
print(f"Validation set size: {num_val} ({num_val/(num_train+num_val+num_test)*100:.2f}%)")
print(f"Test set size: {num_test} ({num_test/(num_train+num_val+num_test)*100:.2f}%)")



Train set size: 64 (71.11%)
Validation set size: 13 (14.44%)
Test set size: 13 (14.44%)


In [7]:
import json

def create_json_file(description, labels, modality, name, root, train_img, train_label, val_img, val_label, test_img, test_label, output_file):
    # Define the data structure
    data = {
        "description": description,
        "labels": labels,
        "modality": modality,
        "name": name,
        "root": root,
        "training": [],
        "validation": [],
        "test": []
    }

    # Populate the training data
    populate_json(train_img, train_label, data, "training")

    # Populate the validation data
    populate_json(val_img, val_label, data, "validation")

    # Populate the test data
    populate_json(test_img, test_label, data, "test")
    
    # Create a JSON file and write the data to it
    with open(output_file, 'w') as json_file:
        json.dump(data, json_file, indent=4)

    print(f"JSON file '{output_file}' created successfully!")

def populate_json(img_list, label_list, data, data_type):
    for i in range(len(img_list)):
        entry = {
            "image": img_list[i],
            "label": label_list[i],
            "monai_name": img_list[i].replace("/", "_").replace("_ToF", "")
        }
        data[data_type].append(entry)




In [8]:
description = "TOPCOW Dataset - 3D TOF MRA images of the Circle of Willis"
labels = {
    "0": "background",
    "1": "vessels"
}
modality = {
    "0": "tof"
}
name = "TOPCOW Dataset"
root = dataset_path


create_json_file(description, labels, modality, name, root, train_img_list, train_label_list, val_img_list, val_label_list, test_img_list, test_label_list, json_out_filepath)

JSON file '/home/falcetta/0_PhD/JOB-VS/JoB-VS/datasets/TOPCOW/dataset.json' created successfully!


### Generate stats json file

In [9]:
from libs.preprocessing.global_features import Global_features_general


Global_features_general(json_out_filepath)


Stats file already exists


In [10]:
dataset_path = "/data/falcetta/Job-VS-TOPCOW/TRAINING/"

In [11]:
!python main.py --batch 32 --data_ver datasets/TOPCOW  --name first_exp

Running on rank 0.
Traceback (most recent call last):
  File "main.py", line 390, in <module>
    main(0, 1, args)
  File "main.py", line 50, in main
    info, model_params = plan_experiment(
  File "/home/falcetta/0_PhD/JOB-VS/JoB-VS/settings.py", line 18, in plan_experiment
    small_size = dataset['small_size']
KeyError: 'small_size'
